In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import f1_score
import numpy as np
from tqdm.notebook import tqdm, trange
torch.manual_seed(1)
all_data=pd.read_csv('WikiLarge_Train.csv')
from sklearn.model_selection import StratifiedShuffleSplit
sss= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(all_data, all_data['label']):
    train_df=all_data.iloc[train_index]
    dev_df=all_data.iloc[test_index]
train_df['label'].value_counts()/len(train_df)
vectorizer = CountVectorizer()


In [2]:
train_df.head()

,original_text,label
401297,"Even when their burrow is underwater , mudskip...",0
347368,The first documented mention of Hungen is in t...,0
325250,"1921 - Karel Appel , Dutch painter",0
3693,Both GQ and In Style had Alba on their June 20...,1
246480,Design The most well-known type of electrode i...,0


In [4]:
X_train = vectorizer.fit_transform(train_df.original_text).todense()
X_dev  = vectorizer.transform(dev_df.original_text).todense()

y_train = list([int(y) for y in train_df['label']])
y_dev  = list([int(y) for y in dev_df['label']])

In [5]:
def to_tensors(instance, label):
    '''Given some vectorized text and the integer label for its class, returns a tuple of these in Tensor format'''
    instance = torch.Tensor(instance)
    label = torch.LongTensor([int(label)])
    return (instance, label)



In [7]:
training_data = [to_tensors(row, y_train[i]) for i, row in enumerate(X_train)]
dev_data = [to_tensors(row, y_dev[i]) for i, row in enumerate(X_dev)]

In [8]:
class LogisticRegression(nn.Module):
    '''A simple multi-class logistic regression network'''
    
   
    def __init__(self, vocab_size, num_classes=2):

       
        super(LogisticRegression, self).__init__()
        
        
        self.betas = nn.Linear(vocab_size, num_classes)

        
    def forward(self, feature_vector):
        '''Takes in the input to the network and then passes it through this
        network's layers(s) and returns the output of the final layer.'''

        
        return self.betas(feature_vector)

In [10]:

loss_function = nn.CrossEntropyLoss()


vocab_size = len(vectorizer.vocabulary_)


model = LogisticRegression(vocab_size)

# We'll use a low learning rate
learning_rate = 5e-5


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 


for epoch in trange(5, desc='Epoch'):
    
 
    for (instance, label) in tqdm(training_data, leave=False, desc='Training Loop for Epoch %d' % epoch):

       
        model.zero_grad()        
                
    
        output = model(instance)
        
       
        loss = loss_function(output, label)
        
    
        loss.backward()
        optimizer.step()

In [11]:
# We'll keep track of our predictions for each instance
dev_preds = []

# Normally, every computation in PyTorch will keep around some state
# information which will let us compute the gradients with the loss
# function. However, if we're doing prediction, we *don't* want that, 
# as it is expensive to keep and compute, and can leave around extra
# state information that we might forget.
#
# To avoid keeping around the state information, we can use 
# torch.no_grad() in a with statement, or we can call model.eval()
# to tell the model to switch into prediction-only model. We can
# switch back into training mode with model.train() too.
with torch.no_grad():
    
    # Let's make a prediction for each item in the dev set
    for (instance, label) in dev_data:
        
        # Note that these outputs are the unnormalized activations
        # of the last layer--not probabilities. However, we don't
        # actually need the probabilities to make predictions!
        outputs = model(instance)
        
        # To make a prediction, we can simply look for which neuron (index)
        # had the highest activation. If we ran these through softmax,
        # that neuron would still be more probable, but this saves us
        # the computation time. 
        #
        # To find the index with the highest value, we'll use np.argmax,
        # which searches and returns that index.
        dev_preds.append(np.argmax(outputs))

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def result_evalation(predict_result, true_result):
    print("accuracy:", accuracy_score(true_result, predict_result))
    print("precision:", precision_score(true_result, predict_result))
    print("recall:", recall_score(true_result, predict_result))
    print("f1:", f1_score(true_result, predict_result))


In [18]:
result_evalation(dev_preds,y_dev)

accuracy: 0.6553614703553519
precision: 0.6733508245877061
recall: 0.6034743383640857
f1: 0.6365005251236888


In [29]:
test_data=pd.read_csv('WikiLarge_Test.csv')

In [20]:
X_test  = vectorizer.transform(test_data.original_text).todense()

In [21]:
test_data = [to_tensors(row, 0) for i, row in enumerate(X_test)]

In [22]:
# We'll keep track of our predictions for each instance
test_preds = []

# Normally, every computation in PyTorch will keep around some state
# information which will let us compute the gradients with the loss
# function. However, if we're doing prediction, we *don't* want that, 
# as it is expensive to keep and compute, and can leave around extra
# state information that we might forget.
#
# To avoid keeping around the state information, we can use 
# torch.no_grad() in a with statement, or we can call model.eval()
# to tell the model to switch into prediction-only model. We can
# switch back into training mode with model.train() too.
with torch.no_grad():
    
    # Let's make a prediction for each item in the dev set
    for (instance, label) in test_data:
        
        # Note that these outputs are the unnormalized activations
        # of the last layer--not probabilities. However, we don't
        # actually need the probabilities to make predictions!
        outputs = model(instance)
        
        # To make a prediction, we can simply look for which neuron (index)
        # had the highest activation. If we ran these through softmax,
        # that neuron would still be more probable, but this saves us
        # the computation time. 
        #
        # To find the index with the highest value, we'll use np.argmax,
        # which searches and returns that index.
        test_preds.append(np.argmax(outputs))

In [34]:
test_preds

[tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 ten

In [35]:
test_preds_new=[]
for i in test_preds:
    test_preds_new.append(int(i))


In [36]:
sum(test_preds_new)

53486

In [37]:
result=pd.DataFrame({'id':list(test_data['id']), 'label':test_preds_new})
result.to_csv('results_LR.csv',index=None)

In [38]:
sum(result['label'])

53486